# GMW Example

This notebook demonstrates how to generate different types of groundwatermonitoringwell (GMW) related sourcedocuments for data exchange with the bronhouderportaal (BRO). Also, it is shown how to validate these documents and how to upload them to the bronhouderportaal api. 

In [1]:
import bro_exchange as brx
from getpass import getpass

Define the token used for authentification with the Bronhouderportaal API below. Note: a token generated in the demo environment should be used.

In [2]:
user = input('user:')
key = getpass('pass:')

acces_token_bro_portal = {
    'user':user,
    'pass':key
    }

## 1. Registrate a new GMW

### 1.1. Generate registration request xml document

Get GMW data:

In [3]:
#%% Inputdata for monitoringtube elements

electrodes = [{'electrodeNumber':1,
               'electrodePackingMaterial':'zand',               
               'electrodeStatus':'gebruiksklaar',               
               'electrodePosition':-2.63,                              
               },
              {'electrodeNumber':2,
               'electrodePackingMaterial':'zand',               
               'electrodeStatus':'gebruiksklaar',               
               'electrodePosition':-3.98,                              
               }]


materialUsed_tube = {'tubePackingMaterial':'bentoniet',
                'tubeMaterial':'peHighDensity',
                'glue':'geen'} 

screen =  {'screenLength':1,
                'sockMaterial':'geen'} 

plainTubePart ={'plainTubePartLength':6.64}   

sedimentSump = {'sedimentSumpLength':0.5} 

geoOhmCables = [{'cableNumber':1,
               'electrodes':electrodes}]

monitoringTubes = [{'tubeNumber':1,
                    'tubeType':'standaardbuis',
                    'artesianWellCapPresent':'nee',
                    'sedimentSumpPresent':'ja',
                    'numberOfGeoOhmCables':1,
                    'tubeTopDiameter':32,
                    'variableDiameter':'nee',
                    'tubeStatus':'gebruiksklaar', 
                    'tubeTopPosition':3.42,
                    'tubeTopPositioningMethod':'RTKGPS10tot20cm',
                    'materialUsed':materialUsed_tube,
                    'screen':screen, 
                    'plainTubePart':plainTubePart},
                   {'tubeNumber':2,
                    'tubeType':'standaardbuis',
                    'artesianWellCapPresent':'nee',
                    'sedimentSumpPresent':'ja',
                    'numberOfGeoOhmCables':1,
                    'tubeTopDiameter':32,
                    'variableDiameter':'nee',
                    'tubeStatus':'gebruiksklaar', 
                    'tubeTopPosition':3.42,
                    'tubeTopPositioningMethod':'RTKGPS10tot20cm',
                    'materialUsed':materialUsed_tube,
                    'screen':screen, 
                    'plainTubePart':plainTubePart,
                    'sedimentSump':sedimentSump, 
                    'geoOhmCables':geoOhmCables}]

#%% Inputdata for GMW_Construction sourcedocument

deliveredLocation = {'X':134750.0,'Y':477800.0,'horizontalPositioningMethod':'DGPS50tot200cm'}

deliveredVerticalPosition = {'localVerticalReferencePoint':'NAP',
                                'offset':0,
                                'verticalDatum':'NAP',
                                'groundLevelPosition':3.18,
                                'groundLevelPositioningMethod':'RTKGPS10tot20cm'
                                }

srcdocdata =  {'objectIdAccountableParty':'B07F0076',
                'deliveryContext':'publiekeTaak',        
                'constructionStandard':'RWSgwmon',        
                'initialFunction':'stand',        
                'numberOfMonitoringTubes':2,        
                'groundLevelStable':'ja',        
                'wellStability':'instabiel',        
                'nitgCode':'B07F0076',        
                'owner':'27376655',        
                'maintenanceResponsibleParty':'27376655',        
                'wellHeadProtector':'geen',        
                'wellConstructionDate':'2015-02-08',        
                'deliveredLocation':deliveredLocation,        
                'deliveredVerticalPosition':deliveredVerticalPosition,             
                'monitoringTubes':monitoringTubes,              
                }    

Generate registration request (GMW_construction) xml file:

In [4]:
# Generate sourcedocument:
reg = brx.gmw_registration_request(srcdoc='GMW_Construction', requestReference = 'register - GMW_Construction - 018. IMBRO - groundwaterMonitoringWell.maintenanceResponsibleParty', deliveryAccountableParty = '52766179', qualityRegime = 'IMBRO', srcdocdata=srcdocdata)
reg.generate()

# Note: a file can be stored optionally, but it's not required
reg.write_xml(filename = 'registration_request_gmw_construction_test.xml', output_dir = 'output\gmw')

### 1.2. Validate registration request xml document

Validate the generated sourcedocument. In this case, the demo environment of the bronhouderportaal is used for validation.

In [5]:
payload = reg.request
validation_info = brx.validate_sourcedoc(payload, acces_token_bro_portal, demo=True)

Check if there are some errors:

In [6]:
print(validation_info['status']+'\n')
try:
    
    print('ERRORS:')
    for error in validation_info['errors']:
        print(error+'\n')
except:
    None

NIET_VALIDE

ERRORS:
Zandvang (SedimentSump) ontbreekt: het moet aanwezig zijn als Monitoringbuis. voorzien van zandvang (MonitoringTube.sedimentSumpPresent) = ja.

Grondwatermonitoringput.put stabiliteit (MonitoringWell.wellStability) mag niet aanwezig zijn:  Grondwatermonitoringput.maaiveld stabiel (MonitoringWell.groundLevelStable) = ja bepaalt dat het moet ontbreken.

Geo-ohmkabel (GeoOhmCable) ontbreekt, {MonitoringWell.monitoringTube.numberOfGeoOhmCables} = 1 waarde bepaalt het aantal keer dat dit gegeven voorkomt.

Er zijn 1 of meer fouten geconstateerd in het brondocument.



There seem to be some errors in the generated sourcedocument. Perform the required adjustments and validate the sourcedocument again:

In [8]:
monitoringTubes[0]['numberOfGeoOhmCables']=0
monitoringTubes[0]['sedimentSumpPresent']='nee'
srcdocdata['groundLevelStable']='nee'
srcdocdata['monitoringTubes']=monitoringTubes


reg = brx.gmw_registration_request(srcdoc='GMW_Construction', requestReference = 'register - GMW_Construction - 018. IMBRO - groundwaterMonitoringWell.maintenanceResponsibleParty', deliveryAccountableParty = '52766179', qualityRegime = 'IMBRO', srcdocdata=srcdocdata)
reg.generate()

payload = reg.request
validation_info = brx.validate_sourcedoc(payload, acces_token_bro_portal, demo=True)

print(validation_info['status']+'\n')
try:
    
    print('ERRORS:')
    for error in validation_info['errors']:
        print(error+'\n')
except:
    None

VALIDE

ERRORS:


The xml document seems to be valid